In [308]:
import pandas as pd
import numpy as np
pred_df = pd.read_csv("/Users/christianhellum/Cand. Merc./Data-Science-Project/data_science_project/Beat the bookies/df_prepro_2.csv", index_col=0)

In [309]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

X = pred_df.drop([ 'FTR', 'Date', 'B365_guess', 'B365_right_or_wrong', ], 1)
Y = pred_df['FTR']

X_test = pred_df.drop(['FTR', 'Date', 'B365_guess', 'B365_right_or_wrong', ], 1)
y_test = pred_df['FTR']

/var/folders/8l/n92rnf315xdf87gkyx0s1n840000gn/T/ipykernel_3020/3654059462.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = pred_df.drop([ 'FTR', 'Date', 'B365_guess', 'B365_right_or_wrong', ], 1)
/var/folders/8l/n92rnf315xdf87gkyx0s1n840000gn/T/ipykernel_3020/3654059462.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_test = pred_df.drop(['FTR', 'Date', 'B365_guess', 'B365_right_or_wrong', ], 1)


In [36]:
import keras
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [310]:
def transform_results(results):
    transformed = []
    for i in range(len(results)):
        if results[i] == 'H':
            transformed.append(0)
        elif results[i] == 'A':
            transformed.append(2)
        else:
            transformed.append(1)
    return np.array(transformed)
            
Y = transform_results(Y)
y_test = transform_results(y_test)

In [311]:
scaler = StandardScaler()

X = scaler.fit_transform(X)
X_test_scaled = scaler.fit_transform(X_test)

In [312]:
# Split our data into train/validation. We are using the 
# most recent full season (18/19) for validation data.
X_train_scaled = X[:-160]
y_train = Y[:-160]
X_test_scaled = X[-160:]
y_test = y_test[-160:]

In [313]:
print('Number of matches in training data:', X_train_scaled.shape[0])
print('Number of matches in test data:', X_test_scaled.shape[0])
print('Number of features:', X_train_scaled.shape[1])

Number of matches in training data: 1700
Number of matches in test data: 160
Number of features: 33


In [314]:
# input dimension is number of features
input_dim = X_train_scaled.shape[1]

activation_func = 'relu'
kernel_init = 'glorot_normal'
batch_size = 32

model = keras.Sequential([
    Dense(48, input_shape=(input_dim,), activation=activation_func),
    Dropout(0.3),
    Dense(16),
    Dropout(0.7),
    Dense(3, activation='softmax')
])

/Users/christianhellum/Library/Caches/pypoetry/virtualenvs/data-science-project-PFmYB0_u-py3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [315]:
es = EarlyStopping(monitor='loss', patience=5, verbose=1)

In [89]:
from keras.optimizers import Adam, SGD

In [316]:
# Define your learning rate
learning_rate = 0.001

# Define your optimizer with the specified learning rate
opt = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# Compile your model with the optimizer
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/Users/christianhellum/Library/Caches/pypoetry/virtualenvs/data-science-project-PFmYB0_u-py3.12/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [317]:
model.fit(X_train_scaled, y_train, batch_size=batch_size, callbacks=[es], epochs=500, verbose=0)

Epoch 31: early stopping


In [318]:
train_loss, train_acc = model.evaluate(X_train_scaled, y_train)
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print('Training loss:', train_loss)
print('Training accuracy:', train_acc)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 922us/step - accuracy: 0.5450 - loss: 0.9632
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5603 - loss: 0.9823 
Training loss: 0.9618840217590332
Training accuracy: 0.5417646765708923
Test loss: 0.9727546572685242
Test accuracy: 0.5625


In [319]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_preds = model.predict(X_test_scaled)
y_pred_argmax = [np.argmax(i) for i in y_preds]
print(classification_report(y_test, y_pred_argmax))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
              precision    recall  f1-score   support

           0       0.55      0.90      0.68        73
           1       0.00      0.00      0.00        38
           2       0.62      0.49      0.55        49

    accuracy                           0.56       160
   macro avg       0.39      0.46      0.41       160
weighted avg       0.44      0.56      0.48       160



/Users/christianhellum/Library/Caches/pypoetry/virtualenvs/data-science-project-PFmYB0_u-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/christianhellum/Library/Caches/pypoetry/virtualenvs/data-science-project-PFmYB0_u-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/christianhellum/Library/Caches/pypoetry/virtualenvs/data-science-project-PFmYB0_u-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precis

In [321]:
funds = 100
wager = 10
favourites = 0
no_bets = 0
min_diff = 0.03

y_preds = model.predict(X_test_scaled)

for i in range(len(X_test_scaled)):
    prediction = np.argmax(y_preds[i])
    print('\nPrediction', prediction)
    print('Actual', y_test[i])
    print('Favourite', np.argmin([X_test['B365H'][i], X_test['B365D'][i], 
                                  X_test['B365A'][i]]))
    print('Prediction proba', y_preds[i])
    print('Home, Draw and Away odds', X_test['B365H'][i],       
          X_test['B365D'][i], X_test['B365A'][i])
    
    if prediction == 0:
        odds_diff = y_preds[i][prediction] - (1/X_test['B365H'][i])
        if odds_diff > min_diff:
            if prediction == np.argmin([X_test['B365H'][i], X_test['B365D'][i], 
                                  X_test['B365A'][i]]):
                favourites +=1
                
            if  prediction == y_test[i]:
                funds += (wager * X_test['B365H'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
            
    elif prediction == 1:
        odds_diff = y_preds[i][prediction] - (1/X_test['B365D'][i])
        if odds_diff > min_diff:
            if prediction == np.argmin([X_test['B365H'][i], X_test['B365D'][i], 
                                  X_test['B365A'][i]]):
                favourites +=1
            if  prediction == y_test[i]:
                funds +=( wager * X_test['B365D'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
    else:
        odds_diff = y_preds[i][prediction] - (1/X_test['B365A'][i])
        if odds_diff >  min_diff:
            if prediction == np.argmin([X_test['B365H'][i], X_test['B365D'][i], 
                                  X_test['B365A'][i]]):
                favourites +=1
            if  prediction == y_test[i]:
                funds += (wager * X_test['B365A'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
   
    print('Funds', funds)
    
print(f'Betted on favourite {favourites} times out of {len(X_test)} matches.')
print(f'No bet placed {no_bets} times')

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

Prediction 0
Actual 2
Favourite 2
Prediction proba [0.46016285 0.29192317 0.24791388]
Home, Draw and Away odds 5.25 3.8 1.65
Funds 90

Prediction 0
Actual 1
Favourite 2
Prediction proba [0.63175976 0.26596034 0.10227986]
Home, Draw and Away odds 4.75 4.2 1.65
Funds 80

Prediction 0
Actual 0
Favourite 0
Prediction proba [0.47146127 0.30381802 0.2247207 ]
Home, Draw and Away odds 1.66 3.75 5.5
Funds 80

Prediction 0
Actual 0
Favourite 2
Prediction proba [0.433768  0.3231253 0.2431066]
Home, Draw and Away odds 2.8 3.2 2.6
Funds 98.0

Prediction 0
Actual 0
Favourite 0
Prediction proba [0.82590353 0.14651744 0.02757906]
Home, Draw and Away odds 2.0 3.2 4.2
Funds 108.0

Prediction 0
Actual 0
Favourite 0
Prediction proba [0.39477965 0.29404914 0.31117117]
Home, Draw and Away odds 1.6 3.8 6.5
Funds 108.0

Prediction 0
Actual 0
Favourite 0
Prediction proba [0.3407702  0.32708058 0.33214918]
Home, Draw and Away odds 2.15 3.2 3.6
Funds 108.0

Prediction 0
Ac